# Import 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import mlflow
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve,
    classification_report,
)
from sklearn.model_selection import GridSearchCV
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
import re
import os
import mlflow.sklearn
import time
from nltk import word_tokenize, pos_tag
import joblib

import optuna
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [4]:
nltk.download("punkt")
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from nltk import FreqDist

# from wordcloud import WordCloud

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/skanderzahi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/skanderzahi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# nltk.download("averaged_perceptron_tagger")
# nltk.download("punkt")
# nltk.download("maxent_ne_chunker")  # Souvent requis aussi
# nltk.download("words")
# nltk.download("averaged_perceptron_tagger_eng")

In [ ]:
print(f"Python version: {sys.version}")
print(f"MLflow version: {mlflow.__version__}")

Python version: 3.13.7 | packaged by Anaconda, Inc. | (main, Sep  9 2025, 19:54:17) [Clang 17.0.6 ]
MLflow version: 3.4.0


In [9]:
df_train = pd.read_csv("dataset/train_50K.csv")
df_val = pd.read_csv("dataset/val_50K.csv")
df_test = pd.read_csv("dataset/test_50K.csv")
df_train.shape, df_val.shape, df_test.shape

((32000, 6), (8000, 6), (10000, 6))

In [ ]:
# df_train = pd.read_csv("dataset/train_50K.csv")
# df_val = pd.read_csv("dataset/val_50K.csv")
# df_test = pd.read_csv("dataset/test_50K.csv")
# df_train.shape, df_val.shape, df_test.shape

In [ ]:
df_train["target"].value_counts(), df_test["target"].value_counts()

(target
 1    32046
 0    31954
 Name: count, dtype: int64,
 target
 1    10023
 0     9977
 Name: count, dtype: int64)

In [ ]:
lemmatizer = WordNetLemmatizer()

# ---------------------------

In [185]:
df_train["target"].value_counts(), df_test["target"].value_counts()

(target
 0    80000
 1    80000
 Name: count, dtype: int64,
 target
 1    20000
 0    20000
 Name: count, dtype: int64)

In [186]:
# Creer un set validation à partir de train
df_train, df_val = train_test_split(
    df_train, test_size=0.2, random_state=42, stratify=df_train["target"]
)
df_train.shape, df_val.shape, df_test.shape

((128000, 6), (32000, 6), (40000, 6))

In [23]:
# Définir le nom de l'expérience
experiment_name = "p7_air_paradis"
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///Users/skanderzahi/Desktop/P7/projet/mlruns/379503310426968982', creation_time=1760449044952, experiment_id='379503310426968982', last_update_time=1760449044952, lifecycle_stage='active', name='p7_air_paradis', tags={'mlflow.experimentKind': 'custom_model_development'}>

# ________

# TF-IDF

## Preprocess 1

Prétraitement aggressif : word_tokenize, stopwords, suppression ponctuation, hashtag, mention...

In [42]:
# Preprocess_1
def preprocess_1(text):
    from nltk.corpus import stopwords

    # Convertir en minuscules
    text = text.lower()

    # Supprimer les URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)

    # Supprimer les mentions et hashtags
    text = re.sub(r"\@\w+|\#", "", text)

    # Supprimer la ponctuation et les caractères spéciaux
    text = re.sub(r"[^\w\s]", "", text)

    # Supprimer les chiffres
    text = re.sub(r"\d+", "", text)

    # Tokenisation
    tokens = word_tokenize(text)

    # Supprimer les stopwords
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Joindre les tokens en une seule chaîne
    text = " ".join(tokens)

    return text

In [59]:
# preprocess_1 train + test
df_train["text_preprocess_1"] = df_train["text"].apply(preprocess_1)
df_test["text_preprocess_1"] = df_test["text"].apply(preprocess_1)

In [136]:
pd.set_option("display.max_colwidth", None)

In [137]:
# Vérification d'un échantillon
print("Exemple de texte avant/après preprocess_1:")
print("Avant:", df_train["text"].iloc[10])
print("Après:", df_train["text_preprocess_1"].iloc[10])

Exemple de texte avant/après preprocess_1:
Avant: my eyes burn 
Après: eye burn


In [83]:
def vectorize_tf_idf(X_train_texts, X_test_texts, vectorizer=None):
    vectorizer = TfidfVectorizer(
        ngram_range=(1, 2), min_df=2, max_df=0.90, max_features=1000
    )
    X_train = vectorizer.fit_transform(X_train_texts)
    X_test = vectorizer.transform(X_test_texts)
    return X_train, X_test, vectorizer

In [61]:
X_train, X_test, vectorizer = vectorize_tf_idf(
    df_train["text_preprocess_1"], df_test["text_preprocess_1"]
)
y_train = df_train["target"]
y_test = df_test["target"]

In [25]:
def calcul_metriques(y_true, y_pred, y_pred_proba):

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred_proba)

    return accuracy, precision, recall, f1, roc_auc

In [47]:
def process_mlflow(run_name, preprocess, X_train, X_test, y_train, y_test, vectorizer):
    with mlflow.start_run(run_name=run_name):

        # Log des paramètres

        # mlflow.log_param("model_type", "LogisticRegression")  # Type de modèle
        # mlflow.log_param("vectorizer", "TF-IDF")  # Type de vectoriseur
        # mlflow.log_param("min_df", 2)  # Seuil de fréquence minimale
        # mlflow.log_param("max_df", 0.90)  # Seuil de fréquence maximale
        # mlflow.log_param("max_features", 1000)  # Nombre maximum de caractéristiques
        # mlflow.log_param("ngram_range", "(1, 2)")  # Plage de n-grammes
        # mlflow.log_param("preprocessing", preprocess)  # Méthode de prétraitement
        # mlflow.log_param("random_state", 42)
        # mlflow.log_param("max_iter", 1000)

        params = {
            "model_type": "LogisticRegression",
            "vectorizer": "TF-IDF",
            "min_df": vectorizer.min_df,
            "max_df": vectorizer.max_df,
            "max_features": vectorizer.max_features,
            "ngram_range": str(vectorizer.ngram_range),
            "preprocessing": preprocess,
            "random_state": 42,
            "max_iter": 1000,
            "train_samples": len(y_train),
            "test_samples": len(y_test),
        }

        mlflow.log_params(params)

        # Mesurer le temps d'exécution
        start_time = time.time()

        # Construction de la régression logistique
        model = LogisticRegression(random_state=42, max_iter=1000)
        model.fit(X_train, y_train)

        training_time = time.time() - start_time

        mlflow.log_metric("training_time_seconds", training_time)

        # Prédire les probabilités pour les ensembles d'entraînement et de test
        y_train_pred = model.predict(X_train)
        y_train_pred_proba = model.predict_proba(X_train)[:, 1]

        y_test_pred = model.predict(X_test)
        y_test_pred_proba = model.predict_proba(X_test)[:, 1]

        # Calcul des métriques
        train_accuracy, train_precision, train_recall, train_f1, train_roc_auc = (
            calcul_metriques(y_train, y_train_pred, y_train_pred_proba)
        )
        test_accuracy, test_precision, test_recall, test_f1, test_roc_auc = (
            calcul_metriques(y_test, y_test_pred, y_test_pred_proba)
        )

        # Affichage des métriques train et test
        print(
            f"Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}"
        )
        print(
            f"Train Precision: {train_precision:.4f}, Test Precision: {test_precision:.4f}"
        )
        print(f"Train Recall: {train_recall:.4f}, Test Recall: {test_recall:.4f}")
        print(f"Train F1 Score: {train_f1:.4f}, Test F1 Score: {test_f1:.4f}")
        print(f"Train ROC AUC: {train_roc_auc:.4f}, Test ROC AUC: {test_roc_auc:.4f}")

        # Train
        mlflow.log_metric("train_accuracy", train_accuracy)
        mlflow.log_metric("train_precision", train_precision)
        mlflow.log_metric("train_recall", train_recall)
        mlflow.log_metric("train_f1", train_f1)
        mlflow.log_metric("train_roc_auc", train_roc_auc)

        # Test
        mlflow.log_metric("test_accuracy", test_accuracy)
        mlflow.log_metric("test_precision", test_precision)
        mlflow.log_metric("test_recall", test_recall)
        mlflow.log_metric("test_f1", test_f1)
        mlflow.log_metric("test_roc_auc", test_roc_auc)

        # Métriques d'overfitting (gap train-test)
        mlflow.log_metric("train_test_accuracy_gap", train_accuracy - test_accuracy)
        mlflow.log_metric("train_test_f1_gap", train_f1 - test_f1)

        # Roc AUC train/test
        plt.figure(figsize=(8, 6))

        for y_true, y_proba, label, color in [
            (y_train, y_train_pred_proba, "Train", "blue"),
            (y_test, y_test_pred_proba, "Test", "red"),
        ]:
            fpr, tpr, _ = roc_curve(y_true, y_proba)
            auc_score = roc_auc_score(y_true, y_proba)
            plt.plot(fpr, tpr, color=color, label=f"{label} (AUC = {auc_score:.3f})")

        plt.plot([0, 1], [0, 1], "k--", alpha=0.5)
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title("ROC Curves - Train/Test")
        plt.legend(loc="lower right")
        plt.grid(True, alpha=0.3)

        plt.savefig("roc_curve.png")
        mlflow.log_artifact("roc_curve.png")
        plt.close()

        # Sauvegarde dans MLflow
        mlflow.sklearn.log_model(model, "logistic_regression_model")
        mlflow.sklearn.log_model(vectorizer, "tfidf_vectorizer")
        print("\nModèle et vectorizer sauvegardés dans MLflow")

        # Sauvegarde locale
        os.makedirs("models", exist_ok=True)

        # Sauvegarder avec joblib
        model_path = f"models/{run_name}_model.joblib"
        joblib.dump(
            model, model_path, compress=3
        )  # compress=3 pour compression optimale
        mlflow.log_artifact(model_path)
        print(f"Modèle sauvegardé (joblib) : {model_path}")

        vectorizer_path = f"models/{run_name}_vectorizer.joblib"
        joblib.dump(vectorizer, vectorizer_path, compress=3)
        mlflow.log_artifact(vectorizer_path)
        print(f"Vectorizer sauvegardé (joblib) : {vectorizer_path}")

        # Log de la figure ROC dans MLflow
        mlflow.log_artifact("roc_curve.png")
        plt.show()
        plt.close()

        # Affichage du temps d'entraînement
        print(f"Training time (seconds): {training_time:.2f}")

        mlflow.set_tags(
            {
                "model_type": "ML_classique",
                "architecture": "logistic_regression",
                "vectorization": "tfidf",
                "dataset": "twitter_50K",
            }
        )

        # Affichage du rapport de classification test
        print("\nClassification Report Test :")
        print(classification_report(y_test, y_test_pred))

In [48]:
process_mlflow(
    run_name="lr_TFIDF_preprocess_1_wordTokenize_lemmatization",
    preprocess="preprocess_1",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    vectorizer=vectorizer,
)

Train Accuracy: 0.7539, Test Accuracy: 0.7402
Train Precision: 0.7405, Test Precision: 0.7289
Train Recall: 0.7829, Test Recall: 0.7668
Train F1 Score: 0.7611, Test F1 Score: 0.7474
Train ROC AUC: 0.8349, Test ROC AUC: 0.8195


2025/11/20 16:21:56 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 16:21:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/20 16:21:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 16:21:58 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/11/20 16:22:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Modèle et vectorizer sauvegardés dans MLflow
Modèle sauvegardé (joblib) : models/lr_TFIDF_preprocess_1_wordTokenize_lemmatization_model.joblib
Vectorizer sauvegardé (joblib) : models/lr_TFIDF_preprocess_1_wordTokenize_lemmatization_vectorizer.joblib
Training time (seconds): 0.08

Classification Report Test :
              precision    recall  f1-score   support

           0       0.75      0.71      0.73      9977
           1       0.73      0.77      0.75     10023

    accuracy                           0.74     20000
   macro avg       0.74      0.74      0.74     20000
weighted avg       0.74      0.74      0.74     20000

🏃 View run lr_TFIDF_preprocess_1_wordTokenize_lemmatization at: http://127.0.0.1:5000/#/experiments/379503310426968982/runs/6ee2101689a54cc2a6d3624e05e76f74
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/379503310426968982


## preprocess 2

Twitter_tokenize, lemmatisation

In [84]:
def tweetTokenize(text):
    tweet_tokenizer = TweetTokenizer(
        preserve_case=False,  # uniformisation en minuscules
        reduce_len=True,  # réduit les répétitions de caractères
        strip_handles=False,  # conserve les @mentions
    )
    tokens = tweet_tokenizer.tokenize(text)
    return tokens

In [86]:
def preprocess_2(text):

    # Supprimer les URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)

    # Supprimer les chiffres
    text = re.sub(r"\d+", "", text)

    # Supprimer les mentions et hashtags
    text = re.sub(r"\@\w+|\#", "", text)

    # Supprimer mots d'1 lettre
    text = re.sub(r"\b[a-z]\b", "", text)

    # Tokenisation
    tokens = tweetTokenize(text)

    # Lemmatisation
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return " ".join(tokens)

In [76]:
# Diviser par deux le dataset pour tester plus rapidement les modèles
df_train = df_train.sample(frac=0.5, random_state=42)
df_test = df_test.sample(frac=0.5, random_state=42)
df_val = df_val.sample(frac=0.5, random_state=42)
df_train.shape, df_val.shape, df_test.shape

((32000, 9), (8000, 7), (10000, 9))

In [87]:
# preprocess_2 train + val + test
df_train["text_preprocess_2"] = df_train["text"].apply(preprocess_2)
df_val["text_preprocess_2"] = df_val["text"].apply(preprocess_2)
df_test["text_preprocess_2"] = df_test["text"].apply(preprocess_2)

In [78]:
df_train["text_preprocess_2"]

33042    very excited for new moon trailer although i w...
7209               rediscovering my love for missy higgins
24367    aww . babe juss called juss cuz he lovez me . ...
19599    brother & million sister get better every time...
31465       going to go see terri , jimmy , & jennifer ...
                               ...                        
21112    i ' sick and tired of twitter because of so ma...
1006     brief and sock . and of course phoebe ' hoodie...
51563    final cut crashed and deleted all my captured ...
42088                    yeah mate im coming in at about :
720                when do we get to hear body control ? ?
Name: text_preprocess_2, Length: 32000, dtype: object

In [79]:
y_train

0        1
1        1
2        0
3        0
4        0
        ..
63995    1
63996    1
63997    1
63998    1
63999    1
Name: target, Length: 64000, dtype: int64

In [23]:
y_train = df_train["target"]
y_test = df_test["target"]
y_val = df_val["target"]

In [195]:
# Vérification d'un échantillon
print("Exemple de texte avant/après preprocess_2:")
print("Avant:", df_train["text"].iloc[0])
print("Après:", df_train["text_preprocess_2"].iloc[0])

Exemple de texte avant/après preprocess_2:
Avant: Good afternoon!  Last night, I had the company of Shermaine through text messaging. Lovely friend I've got to known recently. 
Après: good afternoon ! last night , i had the company of shermaine through text messaging . lovely friend i've got to known recently .


In [88]:
# Vectorisation TF-IDF
X_train, X_test, vectorizer = vectorize_tf_idf(
    df_train["text_preprocess_2"], df_test["text_preprocess_2"]
)

In [198]:
X_train.shape

(64000, 75636)

## Mlflow


In [90]:
process_mlflow(
    run_name="lr_TFIDF_preprocess_2_tweetTokenize_lemmatization_",
    preprocess="preprocess_2",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    vectorizer=vectorizer,
)

2025/11/20 16:32:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Train Accuracy: 0.7671, Test Accuracy: 0.7526
Train Precision: 0.7619, Test Precision: 0.7494
Train Recall: 0.7780, Test Recall: 0.7607
Train F1 Score: 0.7699, Test F1 Score: 0.7550
Train ROC AUC: 0.8468, Test ROC AUC: 0.8338


2025/11/20 16:32:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/20 16:32:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 16:32:30 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/11/20 16:32:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Modèle et vectorizer sauvegardés dans MLflow
Modèle sauvegardé (joblib) : models/lr_TFIDF_preprocess_2_tweetTokenize_lemmatization__model.joblib
Vectorizer sauvegardé (joblib) : models/lr_TFIDF_preprocess_2_tweetTokenize_lemmatization__vectorizer.joblib
Training time (seconds): 0.06

Classification Report Test :
              precision    recall  f1-score   support

           0       0.76      0.74      0.75      9977
           1       0.75      0.76      0.76     10023

    accuracy                           0.75     20000
   macro avg       0.75      0.75      0.75     20000
weighted avg       0.75      0.75      0.75     20000

🏃 View run lr_TFIDF_preprocess_2_tweetTokenize_lemmatization_ at: http://127.0.0.1:5000/#/experiments/379503310426968982/runs/1fbf865d5ad345769141a099f372999e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/379503310426968982


In [ ]:
test_text = (
    "I can't believe you won't do it. I'm so happy! y'all should've seen it, cats"
)
tokens = tweetTokenize(test_text)
print("Avant:", test_text)
print("Avant:", tokens)
print("Après:", [lemmatizer.lemmatize(token) for token in tokens])
tagged = pos_tag(tokens)
lemmas_pos = [lemmatizer.lemmatize(t, tb_to_wn(tag)) for t, tag in tagged]
print("Après avec POS:", lemmas_pos)
stemmer = PorterStemmer()
stemmed = [stemmer.stem(token) for token in tokens]
print("Après avec stemming:", stemmed)

Avant: I can't believe you won't do it. I'm so happy! y'all should've seen it, cats
Avant: ['i', "can't", 'believe', 'you', "won't", 'do', 'it', '.', "i'm", 'so', 'happy', '!', "y'all", "should've", 'seen', 'it', ',', 'cats']
Après: ['i', "can't", 'believe', 'you', "won't", 'do', 'it', '.', "i'm", 'so', 'happy', '!', "y'all", "should've", 'seen', 'it', ',', 'cat']
Après avec POS: ['i', "can't", 'believe', 'you', "won't", 'do', 'it', '.', "i'm", 'so', 'happy', '!', "y'all", "should've", 'see', 'it', ',', 'cat']
Après avec stemming: ['i', "can't", 'believ', 'you', "won't", 'do', 'it', '.', "i'm", 'so', 'happi', '!', "y'all", "should'v", 'seen', 'it', ',', 'cat']


In [ ]:
# from nltk.tokenize import TweetTokenizer
# from nltk import pos_tag

# tknzr = TweetTokenizer()
# tokens = tknzr.tokenize("I can't believe you won't do it. I'm so happy!")
# tags = pos_tag(tokens)
# print(tags)

[('I', 'PRP'), ("can't", 'VBP'), ('believe', 'VB'), ('you', 'PRP'), ("won't", 'WDT'), ('do', 'VB'), ('it', 'PRP'), ('.', '.'), ("I'm", 'NNP'), ('so', 'IN'), ('happy', 'JJ'), ('!', '.')]


## Preprocess 3

In [70]:
def preprocess_3(text):

    # Supprimer les URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)

    # Supprimer les chiffres
    text = re.sub(r"\d+", "", text)

    # Supprimer les mentions et hashtags
    text = re.sub(r"\@\w+|\#", "", text)

    # Tokenisation
    tokens = tweetTokenize(text)

    # Stemmerisation
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    return " ".join(tokens)

In [71]:
# preprocess_3 train + test
df_train["text_preprocess_3"] = df_train["text"].apply(preprocess_3)
df_test["text_preprocess_3"] = df_test["text"].apply(preprocess_3)

In [122]:
# Vérification d'un échantillon
print("Exemple de texte avant/après preprocess_1:")
print("Avant:", df_train["text"].iloc[100])
print("Après:", df_train["text_preprocess_1"].iloc[100])

Exemple de texte avant/après preprocess_1:
Avant: Wow. Just found out that Ronald Takaki committed suicide this week.  RIP, Professor.  http://bit.ly/DbuT1
Après: wow found ronald takaki committed suicide week rip professor


In [72]:
# Vetorisation TF-IDF
X_train, X_test, vectorizer = vectorize_tf_idf(
    df_train["text_preprocess_3"], df_test["text_preprocess_3"]
)

In [73]:
process_mlflow(
    run_name="lr_TFIDF_preprocess_3_tweetTokenize_stemming",
    preprocess="preprocess_3",
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    vectorizer=vectorizer,
)

Train Accuracy: 0.7702, Test Accuracy: 0.7547
Train Precision: 0.7644, Test Precision: 0.7502
Train Recall: 0.7823, Test Recall: 0.7653
Train F1 Score: 0.7732, Test F1 Score: 0.7577
Train ROC AUC: 0.8500, Test ROC AUC: 0.8366


2025/11/20 16:26:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 16:26:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/11/20 16:26:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 16:26:55 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/11/20 16:26:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Modèle et vectorizer sauvegardés dans MLflow
Modèle sauvegardé (joblib) : models/lr_TFIDF_preprocess_3_tweetTokenize_stemming_model.joblib
Vectorizer sauvegardé (joblib) : models/lr_TFIDF_preprocess_3_tweetTokenize_stemming_vectorizer.joblib
Training time (seconds): 0.06

Classification Report Test :
              precision    recall  f1-score   support

           0       0.76      0.74      0.75      9977
           1       0.75      0.77      0.76     10023

    accuracy                           0.75     20000
   macro avg       0.75      0.75      0.75     20000
weighted avg       0.75      0.75      0.75     20000

🏃 View run lr_TFIDF_preprocess_3_tweetTokenize_stemming at: http://127.0.0.1:5000/#/experiments/379503310426968982/runs/12e52b2ef95b402fa85ec88bd9cce184
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/379503310426968982


=> preprocess 2 et 3 presque identiques cependant preprocess 2 est plus rapide en terme d'execution

## Hyperparametres

In [143]:
# Diviser par deux le dataset pour tester plus rapidement les modèles
df_train = df_train.sample(frac=0.5, random_state=42)
df_test = df_test.sample(frac=0.5, random_state=42)
df_val = df_val.sample(frac=0.5, random_state=42)
df_train.shape, df_val.shape, df_test.shape

((64000, 6), (16000, 6), (20000, 6))

In [ ]:
# df_train.to_csv("train_100K.csv", index=False)
# df_val.to_csv("val_100K.csv", index=False)
# df_test.to_csv("test_100K.csv", index=False)

In [ ]:
# def try_param_lr(X_train, X_test, y_train, y_test, model, param_grid):
    
#     from sklearn.model_selection import GridSearchCV

#     model = LogisticRegression(random_state=42)

#     grid_search = GridSearchCV(
#         estimator=model,
#         param_grid=param_grid,
#         scoring="accuracy",
#         cv=5,
#         n_jobs=-1,
#         verbose=2,
#     )

#     grid_search.fit(X_train, y_train)

#     print("Best Hyperparameters:", grid_search.best_params_)

#     best_model = grid_search.best_estimator_

#     # Prédire les probabilités pour les ensembles d'entraînement et de test
#     y_train_pred = best_model.predict(X_train)
#     y_train_pred_proba = best_model.predict_proba(X_train)[:, 1]

#     y_test_pred = best_model.predict(X_test)
#     y_test_pred_proba = best_model.predict_proba(X_test)[:, 1]

#     # Calcul des métriques
#     train_accuracy, train_precision, train_recall, train_f1, train_roc_auc = (
#         calcul_metriques(y_train, y_train_pred, y_train_pred_proba)
#     )
#     test_accuracy, test_precision, test_recall, test_f1, test_roc_auc = (
#         calcul_metriques(y_test, y_test_pred, y_test_pred_proba)
#     )

#     # Affichage des métriques train et test
#     print(f"Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")
#     print(
#         f"Train Precision: {train_precision:.4f}, Test Precision: {test_precision:.4f}"
#     )
#     print(f"Train Recall: {train_recall:.4f}, Test Recall: {test_recall:.4f}")
#     print(f"Train F1 Score: {train_f1:.4f}, Test F1 Score: {test_f1:.4f}")
#     print(f"Train ROC AUC: {train_roc_auc:.4f}, Test ROC AUC: {test_roc_auc:.4f}")

In [ ]:
param_grid = {
    "C": [0.01, 0.1, 1, 10, 100],
    "solver": ["liblinear", "saga"],
    "penalty": ["l1", "l2"],
}
model = LogisticRegression


In [21]:
def process_params_mlflow(
    run_name, X_train, X_test, X_val, y_train, y_test, y_val, param_grid
):
    with mlflow.start_run(run_name=run_name):

        start_time = time.time()

        model = LogisticRegression(random_state=42, max_iter=1000)

        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            scoring="accuracy",
            cv=5,
            n_jobs=-1,
            verbose=2,
            return_train_score=True,
        )
        grid_search.fit(X_train, y_train)

        training_time = time.time() - start_time
        mlflow.log_metric("training_time_seconds", training_time)

        print("Best Hyperparameters:", grid_search.best_params_)

        for param_name, param_value in grid_search.best_params_.items():
            mlflow.log_param(f"best_{param_name}", param_value)

        mlflow.log_metric("best_cv_score", grid_search.best_score_)

        best_model = grid_search.best_estimator_

        # Prédictions
        y_train_pred = best_model.predict(X_train)
        y_train_pred_proba = best_model.predict_proba(X_train)[:, 1]
        y_val_pred = best_model.predict(X_val)
        y_val_pred_proba = best_model.predict_proba(X_val)[:, 1]
        y_test_pred = best_model.predict(X_test)
        y_test_pred_proba = best_model.predict_proba(X_test)[:, 1]

        # Calcul des métriques
        train_accuracy, train_precision, train_recall, train_f1, train_roc_auc = (
            calcul_metriques(y_train, y_train_pred, y_train_pred_proba)
        )
        val_accuracy, val_precision, val_recall, val_f1, val_roc_auc = calcul_metriques(
            y_val, y_val_pred, y_val_pred_proba
        )
        test_accuracy, test_precision, test_recall, test_f1, test_roc_auc = (
            calcul_metriques(y_test, y_test_pred, y_test_pred_proba)
        )
        # Affichage des métriques train et test
        print(
            f"Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}"
        )
        print(
            f"Train Precision: {train_precision:.4f}, Test Precision: {test_precision:.4f}"
        )
        print(f"Train Recall: {train_recall:.4f}, Test Recall: {test_recall:.4f}")
        print(f"Train F1 Score: {train_f1:.4f}, Test F1 Score: {test_f1:.4f}")
        print(f"Train ROC AUC: {train_roc_auc:.4f}, Test ROC AUC: {test_roc_auc:.4f}")

        # Log des paramsètres
        mlflow.log_params(grid_search.best_params_)
        mlflow.log_param("model_type", "LR_GS")
        mlflow.log_param("cv_folds", 5)
        mlflow.log_param("scoring", "accuracy")
        mlflow.log_param("param_grid", str(param_grid))

        # Métriques d'entraînement
        mlflow.log_metric("train_accuracy", train_accuracy)
        mlflow.log_metric("train_precision", train_precision)
        mlflow.log_metric("train_recall", train_recall)
        mlflow.log_metric("train_f1", train_f1)
        mlflow.log_metric("train_roc_auc", train_roc_auc)

        # Métriques de validation
        mlflow.log_metric("val_accuracy", val_accuracy)
        mlflow.log_metric("val_precision", val_precision)
        mlflow.log_metric("val_recall", val_recall)
        mlflow.log_metric("val_f1", val_f1)
        mlflow.log_metric("val_roc_auc", val_roc_auc)

        # Métriques de test
        mlflow.log_metric("test_accuracy", test_accuracy)
        mlflow.log_metric("test_precision", test_precision)
        mlflow.log_metric("test_recall", test_recall)
        mlflow.log_metric("test_f1", test_f1)
        mlflow.log_metric("test_roc_auc", test_roc_auc)

        mlflow.log_metric("best_cv_score", grid_search.best_score_)

        # Overfitting check
        mlflow.log_metric("train_val_accuracy_gap", train_accuracy - val_accuracy)
        mlflow.log_metric("val_test_accuracy_gap", val_accuracy - test_accuracy)
        mlflow.log_metric("train_val_f1_gap", train_f1 - val_f1)
        mlflow.log_metric("val_test_f1_gap", val_f1 - test_f1)

        # Modèle
        mlflow.sklearn.log_model(
            sk_model=best_model,
            name="model",
            registered_model_name="LogisticRegression_Tweets_Sentiment",
        )

        # Courbe ROC
        fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_proba)

        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color="blue", label=f"ROC curve (AUC = {test_roc_auc:.4f})")
        plt.plot([0, 1], [0, 1], color="red", linestyle="--", label="Random")
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title("Receiver Operating Characteristic (ROC) Curve")
        plt.legend(loc="lower right")
        plt.grid(True, alpha=0.3)

        plt.savefig("roc_curve.png")
        mlflow.log_artifact("roc_curve.png")
        plt.close()

        # Affichage du rapport de classification test
        print("\nClassification Report Test :")
        print(classification_report(y_test, y_test_pred))

        mlflow.sklearn.log_model(best_model, "best_lr_model")

        cv_results_df = pd.DataFrame(grid_search.cv_results_)
        cv_results_df.to_csv("cv_results.csv", index=False)
        mlflow.log_artifact("cv_results.csv")

        return best_model, grid_search

In [ ]:
def vectorize_tf_idf(X_train_texts, X_test_texts, X_val_texts, vectorizer=None):
    vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=2, max_df=0.90)
    X_train = vectorizer.fit_transform(X_train_texts)
    X_test = vectorizer.transform(X_test_texts)
    X_val = vectorizer.transform(X_val_texts)
    return X_train, X_test, X_val, vectorizer




In [179]:
X_train, X_test, X_val, vectorizer = vectorize_tf_idf(
    df_train["text_preprocess_1"],
    df_test["text_preprocess_1"],
    df_val["text_preprocess_1"],
)


y_train = df_train["target"]
y_test = df_test["target"]
y_val = df_val["target"]

In [ ]:
param_grid = {
    "C": [0.01, 0.1, 1, 10],
    # "penalty": ["l2", "l1"],
    # "solver": ["liblinear", "saga"],
}

best_model, grid = process_params_mlflow(
    run_name="GridSearch_LogReg_TF-IDF",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    param_grid=param_grid,  
)

In [184]:
param_grid = {
    "C": [1],
    "penalty": ["l2", "l1"],
    # "solver": ["liblinear", "saga"],
}

best_model, grid = process_params_mlflow(
    run_name="GridSearch_LogReg_TF-IDF",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    param_grid=param_grid,  # True si tu as un Model Registry
)

2025/10/16 11:49:13 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/10/16 11:49:13 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] END ..................C=1, penalty=l2, solver=liblinear; total time=   0.3s
[CV] END ..................C=1, penalty=l2, solver=liblinear; total time=   0.3s
[CV] END ..................C=1, penalty=l2, solver=liblinear; total time=   0.3s
[CV] END ..................C=1, penalty=l2, solver=liblinear; total time=   0.3s
[CV] END ..................C=1, penalty=l2, solver=liblinear; total time=   0.3s
[CV] END .......................C=1, penalty=l2, solver=saga; total time=   0.7s
[CV] END .......................C=1, penalty=l2, solver=saga; total time=   0.7s
[CV] END .......................C=1, penalty=l2, solver=saga; total time=   0.7s
[CV] END .......................C=1, penalty=l2, solver=saga; total time=   0.4s
[CV] END .......................C=1, penalty=l2, solver=saga; total time=   0.4s


2025/10/16 11:49:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Best Hyperparameters: {'C': 1, 'penalty': 'l2', 'solver': 'saga'}
Train Accuracy: 0.8420, Test Accuracy: 0.7651
Train Precision: 0.8332, Test Precision: 0.7603
Train Recall: 0.8557, Test Recall: 0.7761
Train F1 Score: 0.8443, Test F1 Score: 0.7681
Train ROC AUC: 0.9176, Test ROC AUC: 0.8453


2025/10/16 11:49:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'LogisticRegression_Tweets_Sentiment' already exists. Creating a new version of this model...
Created version '3' of model 'LogisticRegression_Tweets_Sentiment'.
2025/10/16 11:49:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Classification Report Test :
              precision    recall  f1-score   support

           0       0.77      0.75      0.76      9977
           1       0.76      0.78      0.77     10023

    accuracy                           0.77     20000
   macro avg       0.77      0.77      0.77     20000
weighted avg       0.77      0.77      0.77     20000



2025/10/16 11:49:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/10/16 11:49:22 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/10/16 11:49:22 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [187]:
param_grid = {
    "C": [1, 0.5],
    "penalty": ["l2"],
    "solver": ["saga"],
}

best_model, grid = process_params_mlflow(
    run_name="GridSearch_LogReg_TF-IDF",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    param_grid=param_grid,  # True si tu as un Model Registry
)

2025/10/16 11:50:43 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/10/16 11:50:43 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] END .......................C=1, penalty=l2, solver=saga; total time=   1.0s
[CV] END .......................C=1, penalty=l2, solver=saga; total time=   1.0s
[CV] END .......................C=1, penalty=l2, solver=saga; total time=   1.1s
[CV] END .......................C=1, penalty=l2, solver=saga; total time=   1.1s
[CV] END .....................C=0.5, penalty=l2, solver=saga; total time=   0.9s
[CV] END .....................C=0.5, penalty=l2, solver=saga; total time=   1.0s
[CV] END .......................C=1, penalty=l2, solver=saga; total time=   1.1s
[CV] END .....................C=0.5, penalty=l2, solver=saga; total time=   1.1s
[CV] END .....................C=0.5, penalty=l2, solver=saga; total time=   0.3s
[CV] END .....................C=0.5, penalty=l2, solver=saga; total time=   0.3s
Best Hyperparameters: {'C': 1, 'penalty': 'l2', 'solver': 'saga'}
Train Accuracy: 0.8420, Test Accuracy: 0.7651
Train Precision: 0

2025/10/16 11:50:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/16 11:50:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'LogisticRegression_Tweets_Sentiment' already exists. Creating a new version of this model...
Created version '6' of model 'LogisticRegression_Tweets_Sentiment'.
2025/10/16 11:50:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Classification Report Test :
              precision    recall  f1-score   support

           0       0.77      0.75      0.76      9977
           1       0.76      0.78      0.77     10023

    accuracy                           0.77     20000
   macro avg       0.77      0.77      0.77     20000
weighted avg       0.77      0.77      0.77     20000



2025/10/16 11:50:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/10/16 11:50:49 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/10/16 11:50:49 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


## Optuna => tfidf + RL

In [20]:
# Définir le nom de l'expérience
experiment_name = "p7_air_paradis"
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///Users/skanderzahi/Desktop/P7/projet/mlruns/379503310426968982', creation_time=1760449044952, experiment_id='379503310426968982', last_update_time=1760449044952, lifecycle_stage='active', name='p7_air_paradis', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [22]:
import optuna
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

/opt/anaconda3/envs/glove_test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
df_train.shape, df_val.shape, df_test.shape

((64000, 6), (16000, 6), (20000, 6))

In [24]:
df_train = df_train.sample(frac=0.5, random_state=42)
df_test = df_test.sample(frac=0.5, random_state=42)
df_val = df_val.sample(frac=0.5, random_state=42)
df_train.shape, df_val.shape, df_test.shape

((32000, 6), (8000, 6), (10000, 6))

In [20]:
# preprocess_2 train + val + test
X_train = df_train["text"].apply(preprocess_2)
X_val = df_val["text"].apply(preprocess_2)
X_test = df_test["text"].apply(preprocess_2)

In [53]:
y_train = df_train["target"]
y_val = df_val["target"]
y_test = df_test["target"]

In [43]:
def objective(trial):

    # régulation
    penalty = "l2"  # trial.suggest_categorical("classifier_penalty", ["l1", "l2"])

    solver = "liblinear"  # (  trial.suggest_categorical("solver", ["liblinear", "saga", "lbfgs"]) )

    # if penalty == "l1" and solver == "lbfgs":  # Si l1 stop trial
    #     raise optuna.TrialPruned()

    # Paramètres TF-IDF
    ngram_max = trial.suggest_int("ngram_max", 2, 3)
    min_df = 5  # trial.suggest_int("min_df", 1, 2)
    max_df = 0.95  # trial.suggest_float("max_df", 0.8, 1.0)
    max_features = trial.suggest_int("max_features", 4000, 10000, step=2000)

    # Paramètre C (force de la régularisation)
    C = trial.suggest_float("C", 1e-3, 1e2, log=True)

    # Paramètre max_iter
    max_iter = trial.suggest_int("max_iter", 100, 1000, step=100)

    pipeline = Pipeline(
        [
            (
                "tfidf",
                TfidfVectorizer(
                    ngram_range=(1, ngram_max),
                    min_df=min_df,
                    max_df=max_df,
                    max_features=max_features,
                ),
            ),
            (
                "classifier",
                LogisticRegression(
                    C=C,
                    penalty=penalty,
                    solver=solver,
                    max_iter=max_iter,
                    random_state=42,
                    n_jobs=1,  # 1 job car Optuna parallélise déjà
                ),
            ),
        ]
    )

    # Run MLflow
    with mlflow.start_run(
        nested=True
    ):  # nested=True creer un run enfant pour cet essai (run imbriqués)

        # Logger les paramètres
        params = {
            "trial_number": trial.number,
            "ngram_range": f"(1, {ngram_max})",
            "min_df": min_df,
            "max_df": max_df,
            "max_features": max_features,
            "C": C,
            "penalty": penalty,
            "solver": solver,
            "max_iter": max_iter,
        }

        mlflow.log_params(params)

        cv_scores = cross_val_score(
            pipeline,
            X_train,
            y_train,
            cv=3,
            scoring="accuracy",
            n_jobs=1,  # Pas de parallélisation (conflit avec Optuna)
        )
        # Score mean
        mean_cv_score = cv_scores.mean()
        std_cv_scores = cv_scores.std()

        # Log des metriques
        mlflow.log_metric("cv_accuracy_mean", mean_cv_score)
        mlflow.log_metric("cv_accuracy_std", std_cv_scores)

        # Optuna optimise le score moyen
        score = mean_cv_score

        mlflow.set_tags(
            {
                "trial_number": trial.number,
                "optimizer": "optuna",
                "model": "tfidf_logreg",
            }
        )

    return score

In [44]:
# Etude optuna
study = optuna.create_study(
    study_name="optimisation_tfidf_logreg_50K",
    direction="maximize",  # Maxsimizer accuracy
    storage="sqlite:///optuna_tfidf.db",
    load_if_exists=True,
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=5
    ),  # => Arrête les trials peu prometteurs
)
print(f"{study.study_name} OK")

[I 2025-10-22 16:32:20,953] A new study created in RDB with name: optimisation_tfidf_logreg_50K


optimisation_tfidf_logreg_50K OK


In [48]:
# Optimisation avec Mlflow (run parent)
N_TRIALS = 25
# Run
with mlflow.start_run(run_name=f"tfidf_LR_with_Optuna_50K"):
    mlflow.log_params(
        {
            "optimizer": "optuna",
            "n_trials": N_TRIALS,
            "direction": "maximize",
            "metric": "accuracy",
            "cv_folds": 3,
            "train_samples": len(X_train),
            "val_samples": len(X_val),
            "test_samples": len(X_test),
        }
    )
    # Démarrage de l'optimisation
    start_time = time.time()

    study.optimize(
        objective,
        n_trials=N_TRIALS,
        n_jobs=1,
        show_progress_bar=True,
    )

    elapsed_time = time.time() - start_time

    print(f"Trials complétés: {len(study.trials)}")

    # Meilleurs resultats
    best_trial = study.best_trial
    best_params = study.best_params
    best_value = study.best_value

    print(f"\nMeilleurs paramètres: {best_params}")

    # Logger les meilleurs résultats
    mlflow.log_metric("best_cv_accuracy", best_value)
    mlflow.log_param("best_trial_number", best_trial.number)
    for key, value in best_params.items():
        mlflow.log_param(f"best_{key}", value)

    mlflow.log_metric("optimization_time_minutes", elapsed_time / 60)
    mlflow.log_metric("total_trials", len(study.trials))

    # Entrainement du modèle

    # Recréer le pipeline avec les meilleurs paramètres
    best_pipeline = Pipeline(
        [
            (
                "tfidf",
                TfidfVectorizer(
                    ngram_range=(1, best_params["ngram_max"]),
                    min_df=5,  # best_params["min_df"],
                    max_df=0.95,  # best_params["max_df"],
                    max_features=best_params["max_features"],
                ),
            ),
            (
                "classifier",
                LogisticRegression(
                    C=best_params["C"],
                    penalty="l2",  # best_params["classifier_penalty"],
                    solver="liblinear",  # best_params["classifier_solver"],
                    max_iter=best_params["max_iter"],
                    random_state=42,
                    n_jobs=-1,  # Tous les CPU pour l'entraînement final
                ),
            ),
        ]
    )

    # Entraînement sur train
    model = best_pipeline.fit(X_train, y_train)
    print("Modèle final entraîné avec best_params")

    # Prédictions
    y_train_pred = model.predict(X_train)
    y_train_pred_proba = model.predict_proba(X_train)[:, 1]
    y_val_pred = model.predict(X_val)
    y_val_pred_proba = model.predict_proba(X_val)[:, 1]
    y_test_pred = model.predict(X_test)
    y_test_pred_proba = model.predict_proba(X_test)[:, 1]

    # Calcul des métriques
    train_accuracy, train_precision, train_recall, train_f1, train_roc_auc = (
        calcul_metriques(y_train, y_train_pred, y_train_pred_proba)
    )
    val_accuracy, val_precision, val_recall, val_f1, val_roc_auc = calcul_metriques(
        y_val, y_val_pred, y_val_pred_proba
    )
    test_accuracy, test_precision, test_recall, test_f1, test_roc_auc = (
        calcul_metriques(y_test, y_test_pred, y_test_pred_proba)
    )

    # Affichage des métriques train et test
    print(f"Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")
    print(
        f"Train Precision: {train_precision:.4f}, Test Precision: {test_precision:.4f}"
    )
    print(f"Train Recall: {train_recall:.4f}, Test Recall: {test_recall:.4f}")
    print(f"Train F1 Score: {train_f1:.4f}, Test F1 Score: {test_f1:.4f}")
    print(f"Train ROC AUC: {train_roc_auc:.4f}, Test ROC AUC: {test_roc_auc:.4f}")

    # Métriques d'entraînement
    mlflow.log_metric("train_accuracy", train_accuracy)
    mlflow.log_metric("train_precision", train_precision)
    mlflow.log_metric("train_recall", train_recall)
    mlflow.log_metric("train_f1", train_f1)
    mlflow.log_metric("train_roc_auc", train_roc_auc)

    # Métriques de validation
    mlflow.log_metric("val_accuracy", val_accuracy)
    mlflow.log_metric("val_precision", val_precision)
    mlflow.log_metric("val_recall", val_recall)
    mlflow.log_metric("val_f1", val_f1)
    mlflow.log_metric("val_roc_auc", val_roc_auc)

    # Métriques de test
    mlflow.log_metric("test_accuracy", test_accuracy)
    mlflow.log_metric("test_precision", test_precision)
    mlflow.log_metric("test_recall", test_recall)
    mlflow.log_metric("test_f1", test_f1)
    mlflow.log_metric("test_roc_auc", test_roc_auc)

    # Overfitting check
    mlflow.log_metric("train_val_accuracy_gap", train_accuracy - val_accuracy)
    mlflow.log_metric("val_test_accuracy_gap", val_accuracy - test_accuracy)
    mlflow.log_metric("train_val_f1_gap", train_f1 - val_f1)
    mlflow.log_metric("val_test_f1_gap", val_f1 - test_f1)

    # Courbe ROC pour tarin, val,test
    plt.figure(figsize=(8, 6))
    for y_true, y_proba, label, color in [
        (y_train, y_train_pred_proba, "Train", "blue"),
        (y_val, y_val_pred_proba, "Validation", "green"),
        (y_test, y_test_pred_proba, "Test", "red"),
    ]:
        fpr, tpr, _ = roc_curve(y_true, y_proba)
        auc_score = roc_auc_score(y_true, y_proba)
        plt.plot(fpr, tpr, color=color, label=f"{label} (AUC = {auc_score:.3f})")

    plt.plot([0, 1], [0, 1], "k--", alpha=0.5)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curves - Train/Validation/Test")
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)

    plt.savefig("roc_curve_tfidf.png")
    mlflow.log_artifact("roc_curve_tfidf.png")
    plt.close()

    # Tag Mlflow
    mlflow.set_tags(
        {
            "model_type": "optuna_tfidf_LR",
            "optimizer": "optuna",
            "dataset": "50K",
            "best_trial": best_trial.number,
        }
    )

    # Sauvegarde du modèle
    mlflow.sklearn.log_model(
        best_pipeline,
        artifact_path="model",
        registered_model_name="tfidf_logreg_optuna",
    )

Best trial: 21. Best value: 0.769094:   4%|▍         | 1/25 [00:01<00:40,  1.71s/it]

[I 2025-10-22 16:38:08,609] Trial 50 finished with value: 0.7488126118175815 and parameters: {'ngram_max': 2, 'max_features': 8000, 'C': 15.520580642188301, 'max_iter': 500}. Best is trial 21 with value: 0.7690938022850203.


Best trial: 21. Best value: 0.769094:   8%|▊         | 2/25 [00:04<00:48,  2.10s/it]

[I 2025-10-22 16:38:10,976] Trial 51 finished with value: 0.7659687856768352 and parameters: {'ngram_max': 3, 'max_features': 8000, 'C': 2.733539316698807, 'max_iter': 400}. Best is trial 21 with value: 0.7690938022850203.


Best trial: 21. Best value: 0.769094:  12%|█▏        | 3/25 [00:06<00:49,  2.24s/it]

...

Best trial: 21. Best value: 0.769094: 100%|██████████| 25/25 [01:01<00:00,  2.46s/it]


[I 2025-10-22 16:39:08,420] Trial 74 finished with value: 0.7644687798143472 and parameters: {'ngram_max': 3, 'max_features': 8000, 'C': 4.689725873527641, 'max_iter': 500}. Best is trial 21 with value: 0.7690938022850203.
Trials complétés: 75

Meilleurs paramètres: {'ngram_max': 3, 'max_features': 8000, 'C': 1.935474476346391, 'max_iter': 400}


/opt/anaconda3/envs/projet_7_env/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:1305: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(


Modèle final entraîné avec best_params
Train Accuracy: 0.8417, Test Accuracy: 0.7710
Train Precision: 0.8367, Test Precision: 0.7649
Train Recall: 0.8498, Test Recall: 0.7794
Train F1 Score: 0.8432, Test F1 Score: 0.7721
Train ROC AUC: 0.9191, Test ROC AUC: 0.8498


2025/10/22 16:39:10 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/22 16:39:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'tfidf_logreg_optuna'.
Created version '1' of model 'tfidf_logreg_optuna'.


# Word2vec

In [1]:
from gensim.models import Word2Vec

In [74]:
train_tokens = [text.split() for text in df_train["text_preprocess_2"]]
val_tokens = [text.split() for text in df_val["text_preprocess_2"]]
test_tokens = [text.split() for text in df_test["text_preprocess_2"]]

In [75]:
modele_word2vec = Word2Vec(
    sentences=train_tokens,
    vector_size=100,  #  vector_size 100 dimensions
    window=10,  # contexte
    min_count=2,
    workers=4,
    sg=1,
    seed=42,
    epochs=10,
)
modele_word2vec

In [73]:
# Inspection du modele
print(f"Taille du vocabulaire: {len(modele_word2vec.wv)} mots")
print(f"Dimension des embeddings: {modele_word2vec.vector_size}")

Taille du vocabulaire: 15339 mots
Dimension des embeddings: 100


In [ ]:
words = modele_word2vec.wv

In [56]:
vec1 = words["good"]
vec1

array([-0.14606951,  0.2443309 , -0.16618145, -0.39835092, -0.19266507,
        0.6375418 ,  0.3082946 ,  0.69206226, -0.29387406, -0.03315203,
        0.1407305 ,  0.6681998 ,  0.13545899, -0.04654643, -0.42492375,
       -0.20002839, -0.38407487, -0.18934639, -0.5324323 ,  0.19156584,
        0.29691514, -0.03630893,  0.22086535, -0.10622748,  0.27650416,
       -0.03590391, -0.6126007 ,  0.01843317, -0.5018946 ,  0.30600595,
       -0.25211385, -0.27665663, -0.01966534,  0.24530771, -0.6240829 ,
       -0.44110784, -0.16393135, -0.49064624,  0.0424917 ,  0.14280532,
        0.25607914,  0.38413715,  0.5295669 , -0.2558547 ,  0.41900137,
       -0.08833858,  0.22970605,  0.3063318 ,  0.5627364 , -0.2193433 ,
       -0.07937307, -0.54909104,  0.45931447,  0.0348423 ,  0.01552992,
       -0.22221746,  0.21129425, -0.0902129 , -0.14876099,  0.3967147 ,
       -0.14047666,  0.10062578, -0.09225155,  0.24447569,  0.32669288,
       -0.13255861,  0.2842013 , -0.16739064, -0.01965655,  0.13

In [68]:
vec2 = words["bad"]
# similarité gensim
similarite = words.similarity("good", "bad")
print(f"Similarité gensim {similarite}")
# intrus
print("Intrus entre: good, happy, best, work ")
print(words.doesnt_match(["good", "happy", "best", "work"]))

Similarité gensim 0.5542354583740234
Intrus entre: good, happy, best, work 
work


In [78]:
# Creer les embeddings
def create_embeddings(data_tokens):
    X_data_w2v = []
    for tokens in data_tokens:
        word_vectors = []
        for token in tokens:
            if token in modele_word2vec.wv:
                vector = modele_word2vec.wv[token]
                word_vectors.append(vector)
        if len(word_vectors) > 0:
            tweet_embedding = np.mean(word_vectors, axis=0)
        else:
            tweet_embedding = np.zeros(100)  # Si aucun mot connu vecteur 0
        X_data_w2v.append(tweet_embedding)
    X_data_w2v = np.array(X_data_w2v)
    return X_data_w2v

In [79]:
# Embeddings Train
X_train_w2v = create_embeddings(train_tokens)
# Embeddings Val
X_val_w2v = create_embeddings(val_tokens)
# Embeddings Test
X_test_w2v = create_embeddings(test_tokens)

In [80]:
y_train = df_train["target"].values
y_val = df_val["target"].values
y_test = df_test["target"].values

In [81]:
X_train_w2v

array([[ 0.07551973,  0.00938368, -0.06407581, ...,  0.18204895,
        -0.15318605, -0.01367557],
       [-0.01971032,  0.01715281, -0.04675406, ...,  0.18825826,
        -0.13976344, -0.07234587],
       [ 0.1500515 ,  0.05995679, -0.06817298, ...,  0.1617144 ,
        -0.23004995, -0.06003911],
       ...,
       [ 0.12139874,  0.03876474, -0.17514214, ...,  0.44431865,
        -0.26267782,  0.0842756 ],
       [-0.10128947,  0.12634942, -0.06873481, ...,  0.25397936,
        -0.14696279,  0.02413716],
       [-0.1044376 ,  0.04425542, -0.15892135, ...,  0.17179362,
        -0.03489589, -0.0925146 ]])

In [82]:
# Optuna
def objective_word2vec(trial):
    C = trial.suggest_float("C", 1e-3, 1e2, log=True)
    penalty = "l2"
    solver = trial.suggest_categorical("solver", ["liblinear", "saga", "lbfgs"])

    model = LogisticRegression(
        C=C,
        penalty=penalty,
        solver=solver,
        random_state=42,
        max_iter=1000,
        n_jobs=1,
    )

    # Validation croisée sur les embeddings Word2Vec
    cv_score = cross_val_score(
        model, X_train_w2v, y_train, cv=3, scoring="accuracy", n_jobs=1
    ).mean()  # Moyenne des 3 scores

    return cv_score

In [83]:
# Etude optuna
study = optuna.create_study(
    study_name="Word2Vec_Optuna",
    direction="maximize",
    storage="sqlite:///optuna_word2vec.db",
    load_if_exists=True,
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.MedianPruner(  # Arrête les trials peu prometteurs
        n_startup_trials=5  # Après 5 premiers trials
    ),
)

print(f"{study.study_name} OK")

[I 2025-10-28 13:17:54,527] A new study created in RDB with name: Word2Vec_Optuna


Word2Vec_Optuna OK


In [87]:
# Définir le nom de l'expérience
experiment_name = "p7_air_paradis"
mlflow.set_experiment(experiment_name)

with mlflow.start_run(run_name="Word2Vec_optuna"):

    TRIAL_NUMBER = 25
    EMBEDDING_DIM = X_train_w2v.shape[1]

    # Optimisation
    start_time = time.time()
    study.optimize(
        objective_word2vec, n_trials=TRIAL_NUMBER, n_jobs=1, show_progress_bar=True
    )
    elapsed_time = time.time() - start_time

    # Log de configuration
    mlflow.log_params(
        {
            "optimizer": "optuna",
            "n_trials": TRIAL_NUMBER,
            "embedding_dim": EMBEDDING_DIM,
            "embeddings": "Word2Vec",
            "model": "LogisticRegression",
            "preprocessing": "text_preprocess_2",
            "cv_folds": 3,
            "train_samples": len(X_train_w2v),
            "val_samples": len(X_val_w2v),
            "test_samples": len(X_test_w2v),
        }
    )

    print(f"Meilleur score CV: {study.best_value:.4f}")
    print(f"Meilleurs paramètres: {study.best_params}")

    # Log résultats Optuna
    best_params = study.best_params
    mlflow.log_metrics(
        {
            "best_cv_accuracy": study.best_value,
            "optimization_time_minutes": elapsed_time / 60,
            "total_trials": len(study.trials),
        }
    )
    mlflow.log_params(
        {
            "best_C": best_params["C"],
            "best_solver": best_params["solver"],
        }
    )

    # Modèle final avec best params
    best_model = LogisticRegression(
        C=best_params["C"],
        penalty="l2",
        solver=best_params["solver"],
        random_state=42,
        max_iter=1000,
        n_jobs=-1,
    )

    model = best_model.fit(X_train_w2v, y_train)
    print("Modèle final entraîné avec best_params")

    # Prédictions
    y_train_pred = model.predict(X_train_w2v)
    y_train_pred_proba = model.predict_proba(X_train_w2v)[:, 1]
    y_val_pred = model.predict(X_val_w2v)
    y_val_pred_proba = model.predict_proba(X_val_w2v)[:, 1]
    y_test_pred = model.predict(X_test_w2v)
    y_test_pred_proba = model.predict_proba(X_test_w2v)[:, 1]

    # Métriques
    train_accuracy, train_precision, train_recall, train_f1, train_roc_auc = (
        calcul_metriques(y_train, y_train_pred, y_train_pred_proba)
    )
    val_accuracy, val_precision, val_recall, val_f1, val_roc_auc = calcul_metriques(
        y_val, y_val_pred, y_val_pred_proba
    )
    test_accuracy, test_precision, test_recall, test_f1, test_roc_auc = (
        calcul_metriques(y_test, y_test_pred, y_test_pred_proba)
    )

    # Affichage console
    print(f"Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")
    print(
        f"Train Precision: {train_precision:.4f}, Test Precision: {test_precision:.4f}"
    )
    print(f"Train Recall: {train_recall:.4f}, Test Recall: {test_recall:.4f}")
    print(f"Train F1: {train_f1:.4f}, Test F1: {test_f1:.4f}")
    print(f"Train ROC AUC: {train_roc_auc:.4f}, Test ROC AUC: {test_roc_auc:.4f}")

    # Log métriques
    mlflow.log_metric("train_accuracy", train_accuracy)
    mlflow.log_metric("train_precision", train_precision)
    mlflow.log_metric("train_recall", train_recall)
    mlflow.log_metric("train_f1", train_f1)
    mlflow.log_metric("train_roc_auc", train_roc_auc)

    mlflow.log_metric("val_accuracy", val_accuracy)
    mlflow.log_metric("val_precision", val_precision)
    mlflow.log_metric("val_recall", val_recall)
    mlflow.log_metric("val_f1", val_f1)
    mlflow.log_metric("val_roc_auc", val_roc_auc)

    mlflow.log_metric("test_accuracy", test_accuracy)
    mlflow.log_metric("test_precision", test_precision)
    mlflow.log_metric("test_recall", test_recall)
    mlflow.log_metric("test_f1", test_f1)
    mlflow.log_metric("test_roc_auc", test_roc_auc)

    # Overfitting checks
    mlflow.log_metric("train_val_accuracy_gap", train_accuracy - val_accuracy)
    mlflow.log_metric("val_test_accuracy_gap", val_accuracy - test_accuracy)
    mlflow.log_metric("train_val_f1_gap", train_f1 - val_f1)
    mlflow.log_metric("val_test_f1_gap", val_f1 - test_f1)

    # Courbes ROC train/val/test
    plt.figure(figsize=(8, 6))
    for y_true, y_proba, label, color in [
        (y_train, y_train_pred_proba, "Train", "blue"),
        (y_val, y_val_pred_proba, "Validation", "green"),
        (y_test, y_test_pred_proba, "Test", "red"),
    ]:
        fpr, tpr, _ = roc_curve(y_true, y_proba)
        auc_score = roc_auc_score(y_true, y_proba)
        plt.plot(fpr, tpr, color=color, label=f"{label} (AUC = {auc_score:.3f})")

    plt.plot([0, 1], [0, 1], "k--", alpha=0.5)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curves - Train/Validation/Test (Word2Vec)")
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)

    # Sauvegarde & log de la figure
    plt.savefig("roc_curve_w2v.png")
    mlflow.log_artifact("roc_curve_w2v.png")
    plt.close()

    # Sauvegarde du modèle
    mlflow.sklearn.log_model(
        best_model,
        artifact_path="model",
        registered_model_name=f"w2v_{EMBEDDING_DIM}d_logreg",
    )

    # Tags
    mlflow.set_tags(
        {
            "model_type": "W2V_logreg",
            "embedding_type": "Word2Vec_mean",
            "optimizer": "optuna",
            "dataset": "twitter_50K",
        }
    )

Best trial: 27. Best value: 0.743188:   4%|▍         | 1/25 [00:02<00:51,  2.14s/it]

[I 2025-10-28 14:26:34,561] Trial 50 finished with value: 0.7359374761897947 and parameters: {'C': 0.014558986305195467, 'solver': 'lbfgs'}. Best is trial 27 with value: 0.7431875198938237.


Best trial: 27. Best value: 0.743188:   8%|▊         | 2/25 [00:03<00:38,  1.67s/it]

[I 2025-10-28 14:26:35,900] Trial 51 finished with value: 0.742984391719946 and parameters: {'C': 0.21054831294749465, 'solver': 'saga'}. Best is trial 27 with value: 0.7431875198938237.


Best trial: 27. Best value: 0.743188:  12%|█▏        | 3/25 [00:04<00:34,  1.56s/it]

...

Best trial: 65. Best value: 0.743359:  96%|█████████▌| 24/25 [00:30<00:01,  1.26s/it]

[I 2025-10-28 14:27:03,398] Trial 73 finished with value: 0.7433593946497709 and parameters: {'C': 0.3463409343720759, 'solver': 'saga'}. Best is trial 65 with value: 0.7433593968470021.


Best trial: 65. Best value: 0.743359: 100%|██████████| 25/25 [00:32<00:00,  1.29s/it]


[I 2025-10-28 14:27:04,576] Trial 74 finished with value: 0.7433125225793896 and parameters: {'C': 0.4035032030310661, 'solver': 'saga'}. Best is trial 65 with value: 0.7433593968470021.
Meilleur score CV: 0.7434
Meilleurs paramètres: {'C': 0.3940199293182249, 'solver': 'saga'}
Modèle final entraîné avec best_params
Train Accuracy: 0.7452, Test Accuracy: 0.7358
Train Precision: 0.7484, Test Precision: 0.7410
Train Recall: 0.7397, Test Recall: 0.7267
Train F1: 0.7440, Test F1: 0.7338
Train ROC AUC: 0.8241, Test ROC AUC: 0.8141


2025/10/28 14:27:05 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
python(56407) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
2025/10/28 14:27:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'w2v_100d_logreg'.
Created version '1' of model 'w2v_100d_logreg'.


# GloVe

In [14]:
from gensim.models import KeyedVectors

## 25D

In [6]:
# Charger le fichier GloVe (format texte SANS header) en KeyedVectors 
path = "/Users/skanderzahi/Desktop/glove.twitter.27B/glove.twitter.27B.25d.txt"
kv = KeyedVectors.load_word2vec_format(path, binary=False, no_header=True)
print(kv.vector_size)

25


In [15]:
# Regex précompilées pour gain d'execution
# URL
_re_URL = re.compile(r"(https?://\S+|www\.\S+)")
# Mention et hashtags
_re_mention = re.compile(r"\@\w+|\#")
# Tokenisation
_tweet_tokenizer = TweetTokenizer(
    preserve_case=False, reduce_len=True, strip_handles=False
)

In [16]:
def tweetTokenize(text):
    return _tweet_tokenizer.tokenize(text)

In [17]:
def preprocess_4(text):

    text = _re_URL.sub("", text)

    text = re.sub(r"\@\w+|\#", "", text)  # Mentions et hashtags

    # Supprimer les chiffres seuls
    text = re.sub(r"\b\d+\b", "", text)

    # Tokenisation
    tokens = tweetTokenize(text)

    text = " ".join(tokens)

    return text

In [29]:
text = "Lqsssdqsd 4 coooool;!     #:)qsdqsdqsq https://www.tetx.com @ghgvh #6jhj"
preprocess_4(text)

'lqsssdqsd coool ; ! :) qsdqsdqsq 6jhj'

In [30]:
# preprocess_3 train + val + test
df_train["text_preprocess_glove"] = df_train["text"].apply(preprocess_4)
df_val["text_preprocess_glove"] = df_val["text"].apply(preprocess_4)
df_test["text_preprocess_glove"] = df_test["text"].apply(preprocess_4)

In [84]:
# Vérification d'un échantillon
print("Exemple de texte avant/après text_preprocess_glove:")
idx = 20000
print("Avant:", df_train["text"].iloc[idx])
print("Après:", df_train["text_preprocess_glove"].iloc[idx])

Exemple de texte avant/après text_preprocess_glove:
Avant: I want the rain to go away! There has been too much of it lately 
Après: i want the rain to go away ! there has been too much of it lately


In [32]:
def vectorize_doc(doc_tokens, model):

    vectors = [model[word] for word in doc_tokens if word in model]

    if vectors:  # au moins 1 mot connu
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)  # vecteur nul si aucun mot connu

In [33]:
df_train["glove"] = df_train["text_preprocess_glove"].apply(
    lambda doc: vectorize_doc(doc, kv)
)
df_val["glove"] = df_val["text_preprocess_glove"].apply(
    lambda doc: vectorize_doc(doc, kv)
)
df_test["glove"] = df_test["text_preprocess_glove"].apply(
    lambda doc: vectorize_doc(doc, kv)
)

In [89]:
df_train.shape

(128000, 8)

In [34]:
# Conversion en matrice nympy
X_train_glove = np.stack(df_train["glove"].values)
X_val_glove = np.stack(df_val["glove"].values)
X_test_glove = np.stack(df_test["glove"].values)

In [35]:
X_train_glove

array([[ 0.21352138,  0.11457156,  0.01155368, ...,  0.02365484,
         0.2680077 , -0.33272785],
       [ 0.18230233,  0.04558141,  0.00125615, ..., -0.05999168,
         0.26936516, -0.32877097],
       [ 0.26467943,  0.05912201,  0.00954783, ..., -0.04138768,
         0.1395708 , -0.32912165],
       ...,
       [ 0.24088505,  0.08252805,  0.01570519, ..., -0.04662155,
         0.21732092, -0.34281415],
       [ 0.30595204,  0.09139764,  0.0118228 , ..., -0.06901896,
         0.24238832, -0.35126734],
       [ 0.19691285,  0.02149937, -0.02555363, ...,  0.08218419,
         0.19953941, -0.33092466]])

In [101]:
X_train_glove.shape, X_val_glove.shape, X_test_glove.shape

((64000, 25), (16000, 25), (20000, 25))

In [122]:
y_train = df_train["target"]
y_val = df_val["target"]
y_test = df_test["target"]

In [102]:
# Conversion en DataFrame
glove_df = pd.DataFrame(X_train_glove)
print(glove_df.shape)
glove_df.head(10)

(64000, 25)


,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.683563,0.099428,-0.229688,-0.185015,-0.062875,0.144975,-0.026317,1.920246,-0.073857,0.011684,...,-0.483427,-0.946862,-0.035915,-0.967893,0.326951,-0.170842,-0.380092,0.409900,0.360821,0.349700
1,0.678928,0.097845,-0.170613,-0.139523,-0.178400,0.158301,-0.154635,1.851948,-0.011850,-0.063522,...,-0.353812,-0.854922,-0.024302,-0.952461,0.370596,-0.146648,-0.310857,0.358840,0.478374,0.259727
2,0.657574,0.220625,-0.164633,-0.186143,-0.113194,0.131882,-0.020022,1.886971,-0.021003,0.024397,...,-0.361414,-0.845798,-0.027195,-0.944911,0.324361,-0.182302,-0.356890,0.342174,0.368736,0.323047
3,0.725345,0.185879,-0.118584,-0.164877,0.017729,0.109055,-0.007188,1.975496,-0.168578,-0.008303,...,-0.551350,-1.020258,-0.100154,-0.887534,0.265497,-0.126399,-0.295413,0.409325,0.338336,0.421772
4,0.594795,0.099822,-0.091307,-0.214199,-0.216401,0.152448,0.136176,1.684164,-0.102608,0.264450,...,-0.395616,-0.665527,-0.116331,-0.984068,0.235169,-0.029264,-0.198324,0.260902,0.528516,0.299770
5,0.624171,-0.075879,-0.166146,-0.244411,-0.075554,0.141421,0.287842,1.706552,-0.240711,0.264446,...,-0.508429,-0.890471,-0.078428,-0.950007,-0.000625,-0.109008,-0.269541,0.257833,0.387946,0.329562
6,0.586201,0.206243,-0.283486,-0.212729,-0.146010,0.199475,-0.125786,1.886551,-0.105103,-0.134972,...,-0.316563,-0.848680,-0.007551,-0.918648,0.466142,-0.167460,-0.462902,0.465393,0.299173,0.245711
7,0.699001,0.237430,-0.221662,-0.247956,-0.053430,0.237346,0.040934,1.826516,0.088988,0.237795,...,-0.612978,-0.681874,-0.175957,-0.952437,0.368767,-0.083790,-0.318216,0.439523,0.476513,0.276996
8,0.767231,0.088743,-0.169909,-0.344879,0.136158,-0.025916,0.308373,2.222670,0.033677,0.325735,...,-0.709152,-1.089162,-0.085521,-0.808598,0.123300,-0.382298,-0.364624,0.381978,0.321825,0.428693
9,0.662448,0.113429,-0.239159,-0.304298,-0.204316,0.138914,0.263405,1.665998,-0.003148,0.224594,...,-0.474780,-0.698481,-0.134075,-1.101959,0.286647,-0.093548,-0.369766,0.419327,0.576775,0.254046


In [110]:
param_grid = {
    "C": [0.001, 0.01, 0.1, 1.0, 10.0],  # [0.01, 0.1, 1, 10],
    "penalty": ["l2", "l1"],
    "solver": ["liblinear", "saga"],
}

best_model, grid = process_params_mlflow(
    run_name="glove_25D_GridSearch_LogReg",
    X_train=X_train_glove,
    y_train=y_train,
    X_val=X_val_glove,
    y_val=y_val,
    X_test=X_test_glove,
    y_test=y_test,
    param_grid=param_grid,
)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END ..............C=0.001, penalty=l2, solver=liblinear; total time=   0.1s
[CV] END ..............C=0.001, penalty=l2, solver=liblinear; total time=   0.1s
[CV] END ..............C=0.001, penalty=l2, solver=liblinear; total time=   0.1s
[CV] END ..............C=0.001, penalty=l2, solver=liblinear; total time=   0.2s
[CV] END ..............C=0.001, penalty=l2, solver=liblinear; total time=   0.2s
[CV] END ..............C=0.001, penalty=l1, solver=liblinear; total time=   0.1s
[CV] END ..............C=0.001, penalty=l1, solver=liblinear; total time=   0.1s
[CV] END ..............C=0.001, penalty=l1, solver=liblinear; total time=   0.2s
[CV] END ..............C=0.001, penalty=l1, solver=liblinear; total time=   0.1s
[CV] END ..............C=0.001, penalty=l1, solver=liblinear; total time=   0.1s
[CV] END ...................C=0.001, penalty=l1, solver=saga; total time=   0.9s
[CV] END ...................C=0.001, penalty=l1

2025/10/16 18:05:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/16 18:05:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'LogisticRegression_Tweets_Sentiment' already exists. Creating a new version of this model...
Created version '11' of model 'LogisticRegression_Tweets_Sentiment'.
2025/10/16 18:05:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Classification Report Test :
              precision    recall  f1-score   support

           0       0.56      0.61      0.59      9977
           1       0.58      0.53      0.55     10023

    accuracy                           0.57     20000
   macro avg       0.57      0.57      0.57     20000
weighted avg       0.57      0.57      0.57     20000



2025/10/16 18:05:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


## 100

In [18]:
# Charger le fichier GloVe (format texte SANS header) en KeyedVectors
path = "/Users/skanderzahi/Desktop/glove.twitter.27B/glove.twitter.27B.100d.txt"
kv = KeyedVectors.load_word2vec_format(path, binary=False, no_header=True)

In [37]:
df_train["glove"] = df_train["text_preprocess_glove"].apply(
    lambda doc: vectorize_doc(doc, kv)
)
df_val["glove"] = df_val["text_preprocess_glove"].apply(
    lambda doc: vectorize_doc(doc, kv)
)
df_test["glove"] = df_test["text_preprocess_glove"].apply(
    lambda doc: vectorize_doc(doc, kv)
)
# Conversion en matrice nympy
X_train_glove = np.stack(df_train["glove"].values)
X_val_glove = np.stack(df_val["glove"].values)
X_test_glove = np.stack(df_test["glove"].values)

In [38]:
X_train_glove.shape

(32000, 100)

In [117]:
param_grid = {
    "C": [5.0],  # [0.01, 0.1, 1, 10],
    "penalty": ["l2"],
    "solver": [
        "lbfgs",
    ],
}

best_model, grid = process_params_mlflow(
    run_name="glove_100D_GridSearch_LogReg",
    X_train=X_train_glove,
    y_train=y_train,
    X_val=X_val_glove,
    y_val=y_val,
    X_test=X_test_glove,
    y_test=y_test,
    param_grid=param_grid,
)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ....................C=5.0, penalty=l2, solver=lbfgs; total time=   1.7s
[CV] END ....................C=5.0, penalty=l2, solver=lbfgs; total time=   1.8s
[CV] END ....................C=5.0, penalty=l2, solver=lbfgs; total time=   2.0s
[CV] END ....................C=5.0, penalty=l2, solver=lbfgs; total time=   2.3s
[CV] END ....................C=5.0, penalty=l2, solver=lbfgs; total time=   3.1s
Best Hyperparameters: {'C': 5.0, 'penalty': 'l2', 'solver': 'lbfgs'}


2025/10/17 11:21:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Train Accuracy: 0.5975, Test Accuracy: 0.5919
Train Precision: 0.6047, Test Precision: 0.6011
Train Recall: 0.5665, Test Recall: 0.5521
Train F1 Score: 0.5850, Test F1 Score: 0.5756
Train ROC AUC: 0.6343, Test ROC AUC: 0.6274


2025/10/17 11:21:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'LogisticRegression_Tweets_Sentiment' already exists. Creating a new version of this model...
Created version '14' of model 'LogisticRegression_Tweets_Sentiment'.
2025/10/17 11:21:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Classification Report Test :
              precision    recall  f1-score   support

           0       0.58      0.63      0.61      9977
           1       0.60      0.55      0.58     10023

    accuracy                           0.59     20000
   macro avg       0.59      0.59      0.59     20000
weighted avg       0.59      0.59      0.59     20000



2025/10/17 11:21:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


## Optuna + MLflow

In [19]:
def glove_embedding(df_train, df_val, df_test):

    # Charger le fichier GloVe (format texte SANS header) en KeyedVectors
    path = "/Users/skanderzahi/Desktop/glove.twitter.27B/glove.twitter.27B.100d.txt"
    kv = KeyedVectors.load_word2vec_format(path, binary=False, no_header=True)

    # Regex précompilées pour gain d'execution

    # URL
    _re_URL = re.compile(r"(https?://\S+|www\.\S+)")

    # Mention et hashtags
    _re_mention_hashtag = re.compile(r"\@\w+|\#")

    # Supprimer les chiffres seuls
    _re_digits = re.compile(r"\b\d+\b")

    # Tokenisation
    _tweet_tokenizer = TweetTokenizer(
        preserve_case=False, reduce_len=True, strip_handles=False
    )

    def tweetTokenize(text):
        return _tweet_tokenizer.tokenize(text)

    def preprocess_4(text):
        text = _re_URL.sub("", text)
        text = _re_mention_hashtag.sub("", text)
        text = _re_digits.sub("", text)
        # Tokenisation
        tokens = tweetTokenize(text)
        text = " ".join(tokens)
        return text

    df_train["text_preprocess_glove"] = df_train["text"].apply(preprocess_4)
    df_val["text_preprocess_glove"] = df_val["text"].apply(preprocess_4)
    df_test["text_preprocess_glove"] = df_test["text"].apply(preprocess_4)

    def vectorize_doc(doc_tokens, model):
        vectors = [model[word] for word in doc_tokens if word in model]
        if vectors:  # au moins 1 mot connu
            return np.mean(vectors, axis=0)
        else:
            return np.zeros(model.vector_size)  # vecteur nul si aucun mot connu

    df_train["glove"] = df_train["text_preprocess_glove"].apply(
        lambda doc: vectorize_doc(doc, kv)
    )
    df_val["glove"] = df_val["text_preprocess_glove"].apply(
        lambda doc: vectorize_doc(doc, kv)
    )
    df_test["glove"] = df_test["text_preprocess_glove"].apply(
        lambda doc: vectorize_doc(doc, kv)
    )

    X_train_glove = np.stack(df_train["glove"].values)
    X_val_glove = np.stack(df_val["glove"].values)
    X_test_glove = np.stack(df_test["glove"].values)

    y_train = df_train["target"]
    y_val = df_val["target"]
    y_test = df_test["target"]

    return X_train_glove, X_val_glove, X_test_glove, y_train, y_val, y_test

In [21]:
df_train.shape, df_val.shape, df_test.shape

((64000, 6), (16000, 6), (20000, 6))

In [22]:
df_train = df_train.sample(frac=0.5, random_state=42)
df_test = df_test.sample(frac=0.5, random_state=42)
df_val = df_val.sample(frac=0.5, random_state=42)
df_train.shape, df_val.shape, df_test.shape

((32000, 6), (8000, 6), (10000, 6))

In [23]:
X_train_glove, X_val_glove, X_test_glove, y_train, y_val, y_test = glove_embedding(
    df_train, df_val, df_test
)

In [24]:
X_train_glove.shape

(32000, 100)

In [32]:
# Définir le nom de l'expérience
experiment_name = "p7_air_paradis"
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///Users/skanderzahi/Desktop/P7/projet/mlruns/379503310426968982', creation_time=1760449044952, experiment_id='379503310426968982', last_update_time=1760449044952, lifecycle_stage='active', name='p7_air_paradis', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [33]:
def objective_glove(trial):
    C = trial.suggest_float("C", 1e-3, 1e2, log=True)
    penalty = "l2"
    solver = "liblinear"

    model = LogisticRegression(
        C=C,
        penalty=penalty,
        solver=solver,
        random_state=42,
        n_jobs=1,
    )
    # Validation croisée sur les embeddings GloVe
    cv_score = cross_val_score(
        model, X_train_glove, y_train, cv=3, scoring="accuracy", n_jobs=1
    ).mean()

    return cv_score

In [35]:
# Etude optuna
study = optuna.create_study(
    study_name="GloVe_Optuna",
    direction="maximize",  # Maxsimizer accuracy
    storage="sqlite:///optuna_glove.db",
    load_if_exists=True,
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=5
    ),  # => Arrête les trials peu prometteurs
)
print(f"{study.study_name} OK")

[I 2025-10-22 19:26:14,249] Using an existing study with name 'GloVe_Optuna' instead of creating a new one.


GloVe_Optuna OK


In [ ]:
# mlflow
with mlflow.start_run(run_name="GloVe_optuna"):

    TRIAL_NUMBER = 25
    EMBEDDING_DIM = X_train_glove.shape[1]

    # Optimisation
    start_time = time.time()

    study.optimize(
        objective_glove, n_trials=TRIAL_NUMBER, n_jobs=1, show_progress_bar=True
    )

    elapsed_time = time.time() - start_time

    # Log de configuration
    mlflow.log_params(
        {
            "optimizer": "optuna",
            "n_trials": TRIAL_NUMBER,
            "embedding_dim": EMBEDDING_DIM,
            "embeddings": "GloVe",
            "model": "LogisticRegression",
            "preprocessing": "preprocess_4",
            "cv_folds": 3,
            "train_samples": len(X_train_glove),
            "val_samples": len(X_val_glove),
            "test_samples": len(X_test_glove),
        }
    )

    print(f"Meilleur score CV: {study.best_value:.4f}")
    print(f" Meilleurs paramètres: {study.best_params}")

    # Log des résultats optimisation
    best_params = study.best_params

    mlflow.log_metrics(
        {
            "best_cv_accuracy": study.best_value,
            "optimization_time_minutes": elapsed_time / 60,
            "total_trials": len(study.trials),
        }
    )

    mlflow.log_params(
        {
            "best_C": best_params["C"],
        }
    )

    # solver = "liblinear" if best_params["penalty"] == "l1" else "lbfgs"
    # Construction et entraînement du meilleur modèle
    best_model = LogisticRegression(
        C=best_params["C"],
        penalty="l2",
        solver="liblinear",
        random_state=42,
        n_jobs=-1,
    )

    model = best_model.fit(X_train_glove, y_train)
    print("Modèle final entraîné avec best_params")

    # Prédictions
    y_train_pred = model.predict(X_train_glove)
    y_train_pred_proba = model.predict_proba(X_train_glove)[:, 1]
    y_val_pred = model.predict(X_val_glove)
    y_val_pred_proba = model.predict_proba(X_val_glove)[:, 1]
    y_test_pred = model.predict(X_test_glove)
    y_test_pred_proba = model.predict_proba(X_test_glove)[:, 1]

    # Calcul des métriques
    train_accuracy, train_precision, train_recall, train_f1, train_roc_auc = (
        calcul_metriques(y_train, y_train_pred, y_train_pred_proba)
    )
    val_accuracy, val_precision, val_recall, val_f1, val_roc_auc = calcul_metriques(
        y_val, y_val_pred, y_val_pred_proba
    )
    test_accuracy, test_precision, test_recall, test_f1, test_roc_auc = (
        calcul_metriques(y_test, y_test_pred, y_test_pred_proba)
    )

    # Affichage des métriques train et test
    print(f"Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")
    print(
        f"Train Precision: {train_precision:.4f}, Test Precision: {test_precision:.4f}"
    )
    print(f"Train Recall: {train_recall:.4f}, Test Recall: {test_recall:.4f}")
    print(f"Train F1 Score: {train_f1:.4f}, Test F1 Score: {test_f1:.4f}")
    print(f"Train ROC AUC: {train_roc_auc:.4f}, Test ROC AUC: {test_roc_auc:.4f}")

    # Métriques train
    mlflow.log_metric("train_accuracy", train_accuracy)
    mlflow.log_metric("train_precision", train_precision)
    mlflow.log_metric("train_recall", train_recall)
    mlflow.log_metric("train_f1", train_f1)
    mlflow.log_metric("train_roc_auc", train_roc_auc)

    # Métriques val
    mlflow.log_metric("val_accuracy", val_accuracy)
    mlflow.log_metric("val_precision", val_precision)
    mlflow.log_metric("val_recall", val_recall)
    mlflow.log_metric("val_f1", val_f1)
    mlflow.log_metric("val_roc_auc", val_roc_auc)

    # Métriques test
    mlflow.log_metric("test_accuracy", test_accuracy)
    mlflow.log_metric("test_precision", test_precision)
    mlflow.log_metric("test_recall", test_recall)
    mlflow.log_metric("test_f1", test_f1)
    mlflow.log_metric("test_roc_auc", test_roc_auc)

    # Overfitting check
    mlflow.log_metric("train_val_accuracy_gap", train_accuracy - val_accuracy)
    mlflow.log_metric("val_test_accuracy_gap", val_accuracy - test_accuracy)
    mlflow.log_metric("train_val_f1_gap", train_f1 - val_f1)
    mlflow.log_metric("val_test_f1_gap", val_f1 - test_f1)

    # Courbe ROC pour train, val,test
    plt.figure(figsize=(8, 6))
    for y_true, y_proba, label, color in [
        (y_train, y_train_pred_proba, "Train", "blue"),
        (y_val, y_val_pred_proba, "Validation", "green"),
        (y_test, y_test_pred_proba, "Test", "red"),
    ]:
        fpr, tpr, _ = roc_curve(y_true, y_proba)
        auc_score = roc_auc_score(y_true, y_proba)
        plt.plot(fpr, tpr, color=color, label=f"{label} (AUC = {auc_score:.3f})")

    plt.plot([0, 1], [0, 1], "k--", alpha=0.5)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curves - Train/Validation/Test")
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)

    # Sauvegarder l'image
    plt.savefig("roc_curve_glove.png")
    mlflow.log_artifact("roc_curve_glove.png")
    plt.close()

    # Sauvegarde du modèle
    mlflow.sklearn.log_model(
        best_model,
        artifact_path="model",
        registered_model_name=f"glove_{EMBEDDING_DIM}d_logreg",
    )
    # Taggs
    mlflow.set_tags(
        {
            "model_type": "GloVe_logreg",
            "embedding_type": "GloVe",
            "optimizer": "optuna",
            "dataset": "twitter_50K",
        }
    )

Best trial: 42. Best value: 0.599281:   4%|▍         | 1/25 [00:04<01:58,  4.95s/it]

[I 2025-10-22 21:44:24,238] Trial 133 finished with value: 0.5988749132763425 and parameters: {'C': 80.99505524998528}. Best is trial 42 with value: 0.599281168160101.


Best trial: 42. Best value: 0.599281:   8%|▊         | 2/25 [00:08<01:29,  3.88s/it]

[I 2025-10-22 21:44:27,368] Trial 134 finished with value: 0.5989061417913569 and parameters: {'C': 38.917377782689684}. Best is trial 42 with value: 0.599281168160101.


Best trial: 42. Best value: 0.599281:  12%|█▏        | 3/25 [00:11<01:20,  3.65s/it]

...

Best trial: 42. Best value: 0.599281:  96%|█████████▌| 24/25 [01:26<00:03,  3.31s/it]

[I 2025-10-22 21:45:45,622] Trial 156 finished with value: 0.5988749103465634 and parameters: {'C': 79.3342085248064}. Best is trial 42 with value: 0.599281168160101.


Best trial: 42. Best value: 0.599281: 100%|██████████| 25/25 [01:30<00:00,  3.61s/it]
/opt/anaconda3/envs/glove_test/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1305: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(


[I 2025-10-22 21:45:49,615] Trial 157 finished with value: 0.5991561662066709 and parameters: {'C': 96.86727474485859}. Best is trial 42 with value: 0.599281168160101.
Meilleur score CV: 0.5993
 Meilleurs paramètres: {'C': 98.29105404540472}
Modèle final entraîné avec best_params
Train Accuracy: 0.6009, Test Accuracy: 0.5914
Train Precision: 0.6089, Test Precision: 0.5972
Train Recall: 0.5677, Test Recall: 0.5501
Train F1 Score: 0.5876, Test F1 Score: 0.5727
Train ROC AUC: 0.6384, Test ROC AUC: 0.6277


2025/10/22 21:45:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/22 21:45:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'glove_100d_logreg'.
Created version '1' of model 'glove_100d_logreg'.


# USE

In [1]:
import tensorflow_hub as hub

/opt/anaconda3/envs/p7_use_m2/lib/python3.11/site-packages/tensorflow_hub/__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [5]:
# Chargement du modèle
model_use = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

2025-11-20 20:44:02.594588: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-11-20 20:44:02.594627: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-11-20 20:44:02.594642: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-11-20 20:44:02.594917: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-20 20:44:02.594938: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-11-20 20:44:05.756480: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [11]:
def preprocess_5_use(text):

    text = text.lower()

    text = _re_URL.sub("", text)

    text = re.sub(r"\@\w+|\#", "", text)  # Mentions et hashtags

    # Supprimer les chiffres seuls
    text = re.sub(r"\b\d+\b", "", text)

    # Supprimer espaces multiples
    text = re.sub(r"\s+", " ", text)
    text = text.strip()

    # pas d'utilité de tokenisation pour use (attend txt brut)
    # Tokenisation
    # tokens = tweetTokenize(text)

    return text

In [16]:
# preprocess_4 train + val + test
df_train["text_preprocess_use"] = df_train["text"].apply(preprocess_5_use)
df_val["text_preprocess_use"] = df_val["text"].apply(preprocess_5_use)
df_test["text_preprocess_use"] = df_test["text"].apply(preprocess_5_use)

In [17]:
def vectorize_use_with_batch(text, model, batch_size=1000):
    all_embeddings = []

    for i in range(0, len(text), batch_size):
        batch = text[i : i + batch_size]
        batch_embeddings = model(batch)
        all_embeddings.append(batch_embeddings.numpy())
    return np.vstack(all_embeddings)

In [18]:
list_train = df_train["text_preprocess_use"].tolist()
list_val = df_val["text_preprocess_use"].tolist()
list_test = df_test["text_preprocess_use"].tolist()

In [19]:
X_train_use = vectorize_use_with_batch(list_train, model_use)
X_val_use = vectorize_use_with_batch(list_val, model_use)
X_test_use = vectorize_use_with_batch(list_test, model_use)

In [50]:
X_train_use.shape

(32000, 512)

In [26]:
param_grid = {
    "C": [
        2.0,
        # 5.0,
        # 10.0,
    ],  # [0.001, 0.01, 0.1, 1.0, 10.0],  # [0.01, 0.1, 1, 10],
    "penalty": ["l2"],
    "solver": [
        "liblinear"
        # ,"saga",
        # "lbfgs"
    ],
}

best_model, grid = process_params_mlflow(
    run_name="USE_GridSearch_LogReg_final",
    X_train=X_train_use,
    y_train=y_train,
    X_val=X_val_use,
    y_val=y_val,
    X_test=X_test_use,
    y_test=y_test,
    param_grid=param_grid,
)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/opt/anaconda3/envs/p7_use_m2/lib/python3.11/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)


[CV] END ................C=2.0, penalty=l2, solver=liblinear; total time=   3.7s
[CV] END ................C=2.0, penalty=l2, solver=liblinear; total time=   3.8s
[CV] END ................C=2.0, penalty=l2, solver=liblinear; total time=   3.9s
[CV] END ................C=2.0, penalty=l2, solver=liblinear; total time=   4.3s
[CV] END ................C=2.0, penalty=l2, solver=liblinear; total time=   2.5s
Best Hyperparameters: {'C': 2.0, 'penalty': 'l2', 'solver': 'liblinear'}
Train Accuracy: 0.7878, Test Accuracy: 0.7803
Train Precision: 0.7891, Test Precision: 0.7771
Train Recall: 0.7865, Test Recall: 0.7832
Train F1 Score: 0.7878, Test F1 Score: 0.7801
Train ROC AUC: 0.8692, Test ROC AUC: 0.8581


2025/11/20 20:50:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


MlflowException: When an mlflow-artifacts URI was supplied, the tracking URI must be a valid http or https URI, but it was currently set to file:///Users/skanderzahi/Desktop/P7/projet_7/mlruns. Perhaps you forgot to set the tracking URI to the running MLflow server. To set the tracking URI, use either of the following methods:
1. Set the MLFLOW_TRACKING_URI environment variable to the desired tracking URI. `export MLFLOW_TRACKING_URI=http://localhost:5000`
2. Set the tracking URI programmatically by calling `mlflow.set_tracking_uri`. `mlflow.set_tracking_uri('http://localhost:5000')`

## Optuna + MLflow

In [ ]:
# Définir le nom de l'expérience
experiment_name = "p7_air_paradis"
mlflow.set_experiment(experiment_name)

In [51]:
def objective_use(trial):
    C = trial.suggest_float("C", 1e-3, 1e2, log=True)
    penalty = "l2"
    solver = trial.suggest_categorical("solver", ["liblinear", "saga", "lbfgs"])

    model = LogisticRegression(
        C=C,
        penalty=penalty,
        solver=solver,
        random_state=42,
        n_jobs=1,
    )
    # Validation croisée sur les embeddings GloVe
    cv_score = cross_val_score(
        model, X_train_glove, y_train, cv=3, scoring="accuracy", n_jobs=1
    ).mean()

    return cv_score

In [52]:
# Etude optuna
study = optuna.create_study(
    study_name="USE_Optuna",
    direction="maximize",  # Maxsimizer accuracy
    storage="sqlite:///optuna_use.db",
    load_if_exists=True,
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=5
    ),  # => Arrête les trials peu prometteurs
)
print(f"{study.study_name} OK")

[I 2025-10-22 21:57:26,830] A new study created in RDB with name: USE_Optuna


USE_Optuna OK


In [68]:
# Etude optuna
with mlflow.start_run(run_name="USE_optuna"):

    TRIAL_NUMBER = 25
    EMBEDDING_DIM = X_train_use.shape[1]

    # Optimisation
    start_time = time.time()

    study.optimize(
        objective_use, n_trials=TRIAL_NUMBER, n_jobs=1, show_progress_bar=True
    )

    elapsed_time = time.time() - start_time

    # Log de configuration
    mlflow.log_params(
        {
            "optimizer": "optuna",
            "n_trials": TRIAL_NUMBER,
            "embedding_dim": EMBEDDING_DIM,
            "embeddings": "USE",
            "model": "LogisticRegression",
            "preprocessing": "preprocess_use",
            "cv_folds": 3,
            "train_samples": len(X_train_use),
            "val_samples": len(X_val_use),
            "test_samples": len(X_test_use),
        }
    )

    print(f"Meilleur score CV: {study.best_value:.4f}")
    print(f" Meilleurs paramètres: {study.best_params}")

    # Log des résultats optimisation
    best_params = study.best_params

    mlflow.log_metrics(
        {
            "best_cv_accuracy": study.best_value,
            "optimization_time_minutes": elapsed_time / 60,
            "total_trials": len(study.trials),
        }
    )

    mlflow.log_params(
        {
            "best_C": best_params["C"],
        }
    )

    # solver = "liblinear" if best_params["penalty"] == "l1" else "lbfgs"
    # Construction et entraînement du meilleur modèle
    best_model = LogisticRegression(
        C=best_params["C"],
        penalty="l2",
        solver=best_params["solver"],
        random_state=42,
        n_jobs=-1,
    )

    model = best_model.fit(X_train_use, y_train)
    print("Modèle final entraîné avec best_params")

    # Prédictions
    y_train_pred = model.predict(X_train_use)
    y_train_pred_proba = model.predict_proba(X_train_use)[:, 1]
    y_val_pred = model.predict(X_val_use)
    y_val_pred_proba = model.predict_proba(X_val_use)[:, 1]
    y_test_pred = model.predict(X_test_use)
    y_test_pred_proba = model.predict_proba(X_test_use)[:, 1]

    # Calcul des métriques
    train_accuracy, train_precision, train_recall, train_f1, train_roc_auc = (
        calcul_metriques(y_train, y_train_pred, y_train_pred_proba)
    )
    val_accuracy, val_precision, val_recall, val_f1, val_roc_auc = calcul_metriques(
        y_val, y_val_pred, y_val_pred_proba
    )
    test_accuracy, test_precision, test_recall, test_f1, test_roc_auc = (
        calcul_metriques(y_test, y_test_pred, y_test_pred_proba)
    )

    # Affichage des métriques train et test
    print(f"Train Accuracy: {train_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")
    print(
        f"Train Precision: {train_precision:.4f}, Test Precision: {test_precision:.4f}"
    )
    print(f"Train Recall: {train_recall:.4f}, Test Recall: {test_recall:.4f}")
    print(f"Train F1 Score: {train_f1:.4f}, Test F1 Score: {test_f1:.4f}")
    print(f"Train ROC AUC: {train_roc_auc:.4f}, Test ROC AUC: {test_roc_auc:.4f}")

    # Métriques d'entraînement
    mlflow.log_metric("train_accuracy", train_accuracy)
    mlflow.log_metric("train_precision", train_precision)
    mlflow.log_metric("train_recall", train_recall)
    mlflow.log_metric("train_f1", train_f1)
    mlflow.log_metric("train_roc_auc", train_roc_auc)

    # Métriques de validation
    mlflow.log_metric("val_accuracy", val_accuracy)
    mlflow.log_metric("val_precision", val_precision)
    mlflow.log_metric("val_recall", val_recall)
    mlflow.log_metric("val_f1", val_f1)
    mlflow.log_metric("val_roc_auc", val_roc_auc)

    # Métriques de test
    mlflow.log_metric("test_accuracy", test_accuracy)
    mlflow.log_metric("test_precision", test_precision)
    mlflow.log_metric("test_recall", test_recall)
    mlflow.log_metric("test_f1", test_f1)
    mlflow.log_metric("test_roc_auc", test_roc_auc)

    # Overfitting check
    mlflow.log_metric("train_val_accuracy_gap", train_accuracy - val_accuracy)
    mlflow.log_metric("val_test_accuracy_gap", val_accuracy - test_accuracy)
    mlflow.log_metric("train_val_f1_gap", train_f1 - val_f1)
    mlflow.log_metric("val_test_f1_gap", val_f1 - test_f1)

    # Courbe ROC pour tarin, val,test
    plt.figure(figsize=(8, 6))
    for y_true, y_proba, label, color in [
        (y_train, y_train_pred_proba, "Train", "blue"),
        (y_val, y_val_pred_proba, "Validation", "green"),
        (y_test, y_test_pred_proba, "Test", "red"),
    ]:
        fpr, tpr, _ = roc_curve(y_true, y_proba)
        auc_score = roc_auc_score(y_true, y_proba)
        plt.plot(fpr, tpr, color=color, label=f"{label} (AUC = {auc_score:.3f})")

    plt.plot([0, 1], [0, 1], "k--", alpha=0.5)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curves - Train/Validation/Test")
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)

    # Sauvegarder l'image
    plt.savefig("roc_curve_use.png")
    mlflow.log_artifact("roc_curve_use.png")
    plt.close()

    # Sauvegarde du modèle
    mlflow.sklearn.log_model(
        best_model,
        name="model",
        registered_model_name=f"use_{EMBEDDING_DIM}d_logreg",
    )
    # Taggs
    mlflow.set_tags(
        {
            "model_type": "USE_logreg",
            "embedding_type": "USE",
            "optimizer": "optuna",
            "dataset": "twitter_50K",
        }
    )

🏃 View run USE_optuna at: http://127.0.0.1:5000/#/experiments/379503310426968982/runs/3784d41def764788a39dd6569ffa1de3
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/379503310426968982


NameError: name 'X_train_use' is not defined